# Importação e preparação dos dados

In [11]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-unicos.csv', sep = '|')
df.head()

,acordao,arquivo,areas,texto
0,297/2016-P,547240.txt,Responsabilidade,TRIBUNAL DE CONTAS DA UNIÃO\tTC 010.084/2015-0...
1,366/2016-P,549518.txt,Finanças Públicas,TRIBUNAL DE CONTAS DA UNIÃO\tTC 005.933/2014-5...
2,944/2016-P,554399.txt,Responsabilidade,TRIBUNAL DE CONTAS DA UNIÃO\tTC 042.038/2012-0...
3,30/2016-P,545010.txt,Direito Processual,TRIBUNAL DE CONTAS DA UNIÃO\tTC 000.742/2014-7...
4,55/2016-P,544046.txt,Pessoal,;-;;Wania Lucia Pasquarelli do NascimentoTCUWa...


In [12]:
df.shape

(9739, 4)

In [13]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [14]:
y = lbArea.transform(df['areas'])
y.shape

(9739, 10)

In [15]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 320000
limite_texto = 40000
dim_vetor = 50

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['texto'])

sequences = tokenizer.texts_to_sequences(df['texto'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 318318 unique tokens.


In [16]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (9739, 40000)


In [17]:
x.shape, y.shape

((9739, 40000), (9739, 10))

In [18]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos-50.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [19]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Vocabulario: 318318
Encontrados no modelo: 89007 = 27.961660980528904


# Treinamento

In [10]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0315 23:34:20.693207 139935880312640 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0315 23:34:20.927626 139935880312640 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0315 23:34:20.958457 139935880312640 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0315 23:34:20.981295 139935880312640 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40000, 50)         16000000  
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               22912     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 16,024,202
Trainable params: 16,024,202
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 9933s 1s/step - loss: 1.9168 - categorical_accuracy: 0.3341 - val_loss: 2.1780 - val_categorical_accuracy: 0.1961
Epoch 2/20
7791/7791 [==============================] - 9906s 1s/step - loss: 1.8346 - categorical_accuracy: 0.3695 - val_loss: 1.9239 - val_categorical_accuracy: 0.2464
Epoch 

KeyboardInterrupt: 

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40000, 50)         16000000  
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               68736     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 16,070,026
Trainable params: 16,070,026
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
  32/7791 [..............................] - ETA: 14:08:49 - loss: 2.6714 - categorical_accuracy: 0.1250

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)